In [1]:
%matplotlib inline
import gc, math, pickle, datetime, os, random
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.display import display
from statistics import *
from datetime import datetime
import seaborn as sns
from sklearn.preprocessing import normalize
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from sklearn.metrics import mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MultiLabelBinarizer
import lightgbm as lgb

In [2]:
def reduce_mem_usage(df, verbose=True):
    """
    :param df: Dataframe with columns unprocessed so they use more memory than needed
    
    :returns:
        df -> Dataframe with lower memory use
    """
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Predictions lower than zero are turned zero
def fix_predictions(y):
    """
    :param y: Column with predictions
    """
    y[y < 0] = 0

In [34]:
# load test set
df_test = pd.read_csv('E:/data/Kaggle Energy Prediction/test.csv')

In [35]:
df_test.dtypes

row_id          int64
building_id     int64
meter           int64
timestamp      object
dtype: object

In [36]:
df_test['timestamp'] = pd.to_datetime(df_test['timestamp']) #need to convert to float for LightLGB
df_test['hour'] = df_test['timestamp'].dt.hour
df_test['weekday'] = df_test.timestamp.dt.weekday
df_test['month'] = df_test.timestamp.dt.month
df_test['is_weekend'] = np.where(df_test.weekday > 5, 1, 0)

In [37]:
df_building_meta = pd.read_csv('E:/data/Kaggle Energy Prediction/building_metadata.csv')
df_weather_test = pd.read_csv('E:/data/Kaggle Energy Prediction/weather_test.csv')

In [38]:
df_test.dtypes

row_id                  int64
building_id             int64
meter                   int64
timestamp      datetime64[ns]
hour                    int64
weekday                 int64
month                   int64
is_weekend              int32
dtype: object

In [39]:
df_building_meta.dtypes

site_id          int64
building_id      int64
primary_use     object
square_feet      int64
year_built     float64
floor_count    float64
dtype: object

In [40]:
df_weather_test.dtypes

site_id                 int64
timestamp              object
air_temperature       float64
cloud_coverage        float64
dew_temperature       float64
precip_depth_1_hr     float64
sea_level_pressure    float64
wind_direction        float64
wind_speed            float64
dtype: object

In [41]:
df_test = df_test.merge(df_building_meta, how = 'left', on = 'building_id')
df_test.dtypes

row_id                  int64
building_id             int64
meter                   int64
timestamp      datetime64[ns]
hour                    int64
weekday                 int64
month                   int64
is_weekend              int32
site_id                 int64
primary_use            object
square_feet             int64
year_built            float64
floor_count           float64
dtype: object

In [42]:
df_test.head()

,row_id,building_id,meter,timestamp,hour,weekday,month,is_weekend,site_id,primary_use,square_feet,year_built,floor_count
0,0,0,0,2017-01-01,0,6,1,1,0,Education,7432,2008.0,NaN
1,1,1,0,2017-01-01,0,6,1,1,0,Education,2720,2004.0,NaN
2,2,2,0,2017-01-01,0,6,1,1,0,Education,5376,1991.0,NaN
3,3,3,0,2017-01-01,0,6,1,1,0,Education,23685,2002.0,NaN
4,4,4,0,2017-01-01,0,6,1,1,0,Education,116607,1975.0,NaN


In [43]:
df_weather_test['timestamp'] = pd.to_datetime(df_weather_test['timestamp'])
df_test = df_test.merge(df_weather_test, how = 'left', on = ['site_id','timestamp'])

In [44]:
df_test.head()

,row_id,building_id,meter,timestamp,hour,weekday,month,is_weekend,site_id,primary_use,square_feet,year_built,floor_count,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,0,0,2017-01-01,0,6,1,1,0,Education,7432,2008.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
1,1,1,0,2017-01-01,0,6,1,1,0,Education,2720,2004.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
2,2,2,0,2017-01-01,0,6,1,1,0,Education,5376,1991.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
3,3,3,0,2017-01-01,0,6,1,1,0,Education,23685,2002.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6
4,4,4,0,2017-01-01,0,6,1,1,0,Education,116607,1975.0,NaN,17.8,4.0,11.7,NaN,1021.4,100.0,3.6


In [45]:
df_test.dtypes

row_id                         int64
building_id                    int64
meter                          int64
timestamp             datetime64[ns]
hour                           int64
weekday                        int64
month                          int64
is_weekend                     int32
site_id                        int64
primary_use                   object
square_feet                    int64
year_built                   float64
floor_count                  float64
air_temperature              float64
cloud_coverage               float64
dew_temperature              float64
precip_depth_1_hr            float64
sea_level_pressure           float64
wind_direction               float64
wind_speed                   float64
dtype: object

In [46]:
#create adjusted hours
# see part1 for below file
df_weather_train_study = pd.read_csv('E:/data/Kaggle Energy Prediction/site_temp_trend.csv')
df_weather_train_study['max_temp_of_the_area'] = df_weather_train_study.groupby('site_id').air_temperature.transform(np.max)
df_site_time_adj = df_weather_train_study.loc[df_weather_train_study.air_temperature == df_weather_train_study.max_temp_of_the_area].reset_index()
df_site_time_adj['time_adj'] = df_site_time_adj['hour'] - 12

df_test = df_test.merge(df_site_time_adj[['site_id','time_adj']], how = 'left', on = 'site_id')
df_test['hour_adj'] = np.where(df_test.hour - df_test.time_adj < 0 ,24 + df_test.hour - df_test.time_adj ,df_test.hour - df_test.time_adj)

df_test.dtypes

MemoryError: 